AuxTel test LTS-337-030 (Auxiliary Telescope Dome Rotation Requirement)

In this notebook, we check the speed of the dome rotation. As part of the dome refurbishment effort, the minimum dome rotation speed shall be increased to a minimum of Aux_Tel_dome_rot_spd. The contractor shall supply the required hardware to support 4 drive motors. 

| Description | Value       | Unit          |   Name     |
| :---        |    :----:   |       :----:  |       ---: |
|The azimuth rotational speed for the Auxiliary Telescope Dome shall be a minimum of: | 4.0       | degrees/sec   |Aux_Tel_dome_rot_spd|

An azimuth angular rotation of three different rotation angles will be commanded and timed. The ellapsed time, angular rotation angle and motion speed will get printed in the notebook. This calculated average speed includes the acceleration and decceleration of the dome. The instantaneous speed of the dome will be calculated with the gradient at each position and an average slew speed will be obtained from the slope of a linear regression of the position vs time in the uniform motion interval of the move. Results will be saved in a text file (Attached AuxTel_LTS-337-030_20220311.txt)

This notebook is divided into 6 sections:
1. Notebook and ATCS Setup 
2. Test Long Angular Rotation - 210 deg 
3. Test Medium Angular Rotation - 90 degrees
4. Test Short Angular Rotation - 30 degrees 
5. Shutdown 
6. Offline analysis of observing run dome speed (2022-02-16) 
7. Conclusion. 

# Notebook and ATCS Setup. 

In [ ]:
import sys, time, os, asyncio
import pandas as pd

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts import salobj
from astropy.time import Time, TimeDelta
from datetime import datetime, date

from lsst_efd_client import EfdClient

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

In [ ]:
# Get EFD client
client = EfdClient('summit_efd')

In [ ]:
domain = salobj.Domain()

In [ ]:
atcs = ATCS(domain)
await asyncio.gather(atcs.start_task)

In [ ]:
# Enable components ATCS
await atcs.enable({"atdome": "current", "ataos": "current", "athexapod": "current"})

In [ ]:
# Disable Dome following 
await atcs.disable_dome_following()

In [ ]:
await atcs.slew_dome_to(20.0)

In [ ]:
await atcs.home_dome()

In [ ]:
# Enter the filename where to save the results.
filename = 'AuxTel_LTS-337-014_' + date.today().strftime("%y%m%d") + '.txt'
print(filename)

# Test Long Rotation - 210 degrees

In [ ]:
# Long Rotation 210 degrees
test_type = 'Long Rotation - 210 deg'
initial_position = 260
final_position = 50

print(f'The initial position is {initial_position} degrees \n'
      f'The final position is {final_position} degrees \n'
      f'Results will be saved in {filename}')

In [ ]:
 # Move to initial position 
print(f'Moving to initial azimuth position = '
      f'{initial_position} degrees on {time.asctime()}')

await atcs.slew_dome_to(az=initial_position)

# Get dome azimuth initial position and move start time 
start_time = Time(Time.now(), format='fits', 
                         scale='utc')

dome_azimuth = await client.select_top_n(
    "lsst.sal.ATDome.position", 'azimuthPosition', 1)
az_origin = dome_azimuth.iat[0,0]
print(f'Initial azimuth position is = {az_origin} degrees '
      f'on {start_time}')

# Move to final position
print(f'Moving to final azimuth position = '
      f'{final_position} degrees on {time.asctime()}')
await atcs.slew_dome_to(az=final_position)

# Get dome azimuth final position and move end time
end_time = Time(Time.now(), format='fits', scale='utc')

dome_azimuth = await client.select_top_n(
    "lsst.sal.ATDome.position", 'azimuthPosition', 1)
az_final = dome_azimuth.iat[0,0]
print(f'Final azimuth position is = {az_final} degrees ' 
      f'on {end_time}')


# Some quick calculations and print results. 
duration_time= TimeDelta(end_time - start_time, 
                         format='sec')
rotation_angle = abs(az_origin - az_final)

long_speed = rotation_angle/duration_time.value

## Results

In [ ]:
print(f'######################################################### \n'
          f'The move lasted {duration_time.value:.2f} seconds \n'
          f'for an angular rotation of {rotation_angle:.2f} degrees \n'
          f'with an average rotational speed of {long_speed:.2f} degrees/sec\n'
          f'#########################################################')
    

## Dome azimuth speed

In [ ]:
# Get EFD azimuth position time series between start_time and end_time
df_longslew = await client.select_time_series("lsst.sal.ATDome.position", 'azimuthPosition', 
                                                    start_time, end_time)

In [ ]:
azimuth = np.array(df_longslew.values.tolist())[:,0]
times=(df_longslew.index - df_longslew.index[0]).total_seconds()
speed_deriv = np.gradient(azimuth, times)


In [ ]:
%matplotlib inline
fig, ax = plt.subplots(num='Long Slew', figsize=(20, 6))

ax.plot(times, azimuth, 'C0o-', 
        label='Azimuth Position', lw=0.5, zorder=3)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Azimuth Position [degrees]')
ax.set_title(f'Long Slew Dome Rotation test')

ax2 = ax.twinx()
ax2.plot(times, speed_deriv, 'x', color = 'green',
                label='Azimuth Speed', lw=0.5)
ax2.set_ylabel('Azimuth Speed [deg/s]')
ax2.set_ylim(-4.5,4.5)
plt.hlines(y=[-4, 4], xmin=[times[0]], xmax=[times[-1]], colors='orange', 
           linestyles='--', lw=2, label='Minimum Dome Rotation Speed Requirement')
fig.legend()
plt.show()


The dome azimuth speeed (green) is smaller than the required 4 deg/s along the move (orange dotted horizontal lines). 
If we remove the acceleration/decceleration parts of the move and focus in the uniform motion:


In [ ]:
fig, ax = plt.subplots(num='Long Slew', figsize=(20, 6))
[i1,i2] = [140,180]

ax.plot(times[i1:i2], azimuth[i1:i2], 'C0o-', 
        label='Azimuth Position', lw=0.5, zorder=3)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Azimuth Position [degrees]')
ax.set_title(f'Long Slew Dome Rotation test - Focus on uniform motion')

ax2 = ax.twinx()
ax2.plot(times[i1:i2], speed_deriv[i1:i2], 'x', color = 'green',
                label='Azimuth Speed', lw=0.5)
ax2.set_ylabel('Azimuth Speed [deg/s]')
ax2.set_ylim(-5,5)
plt.hlines(y=[-4, 4], xmin=[times[i1]-2], xmax=[times[i2]+2], colors='orange', 
           linestyles='--', lw=2, label='Minimum Dome Rotation Speed Requirement')
fig.legend()
plt.show()

In [ ]:
max_speed_longslew = np.max(abs(speed_deriv[i1:i2]))
print(f'The maximum slew speed is {max_speed_longslew:0.2f} deg/s')

In [ ]:
# Linear regression
model = np.polyfit(times[i1:i2], azimuth[i1:i2], 1)
slope_longslew = model[0]
print(f'The average slew speed is {abs(slope_longslew):0.2f} deg/s')

In [ ]:
# Save test results in a file (See file attached)
with open(filename, 'a') as f:
    f.write('\n{} test on {} \n'.format(test_type, time.asctime()))
    f.write('Start Time, End Time, Rotation angle (degrees), ' 
            'Rotational speed (degrees/sec) \n')
    f.write(f'{start_time.value}, {end_time.value},' 
            f'{round(rotation_angle,2)},{round(long_speed,2)} \n')
    f.write(f'The maximum slew speed is {max_speed_longslew:0.2f} deg/s \n')
    f.write(f'The average slew speed is {abs(slope_longslew):0.2f} deg/s \n')
f.close()
print(f'Start and end move times, rotation angle (degrees) and rotational speeds ' 
      f'(degrees/sec) are saved in {filename}')

# Test Medium Rotation - 90 degrees

In [ ]:
# # Medium rotation - 90 degrees
test_type = 'Medium Rotation - 90 deg'
initial_position = 90
final_position = 180

print(f'The initial position is {initial_position} degrees \n'
      f'The final position is {final_position} degrees \n'
      f'Results will be saved in {filename}')

In [ ]:
 # Move to initial position 
print(f'Moving to initial azimuth position = '
      f'{initial_position} degrees on {time.asctime()}')

await atcs.slew_dome_to(az=initial_position)

# Get dome azimuth initial position and move start time 
start_time = Time(Time.now(), format='fits', 
                         scale='utc')

dome_azimuth = await client.select_top_n(
    "lsst.sal.ATDome.position", 'azimuthPosition', 1)
az_origin = dome_azimuth.iat[0,0]
print(f'Initial azimuth position is = {az_origin} degrees '
      f'on {start_time}')

# Move to final position
print(f'Moving to final azimuth position = '
      f'{final_position} degrees on {time.asctime()}')
await atcs.slew_dome_to(az=final_position)

# Get dome azimuth final position and move end time
end_time = Time(Time.now(), format='fits', scale='utc')

dome_azimuth = await client.select_top_n(
    "lsst.sal.ATDome.position", 'azimuthPosition', 1)
az_final = dome_azimuth.iat[0,0]
print(f'Final azimuth position is = {az_final} degrees ' 
      f'on {end_time}')


# Some quick calculations and print results. 
duration_time= TimeDelta(end_time - start_time, 
                         format='sec')
rotation_angle = abs(az_origin - az_final)

med_speed = rotation_angle/duration_time.value

## Results

In [ ]:
print(f'######################################################### \n'
          f'The move lasted {duration_time.value:.2f} seconds \n'
          f'for an angular rotation of {rotation_angle:.2f} degrees \n'
          f'with an average rotational speed of {med_speed:.2f} degrees/sec\n'
          f'#########################################################')
    

## Dome azimuth speed

In [ ]:
# Get EFD azimuth position time series between start_time and end_time
df_mediumslew = await client.select_time_series("lsst.sal.ATDome.position", 'azimuthPosition', 
                                                    start_time, end_time)

In [ ]:
azimuth = np.array(df_mediumslew.values.tolist())[:,0]
times=(df_mediumslew.index - df_mediumslew.index[0]).total_seconds()
speed_deriv = np.gradient(azimuth, times)


In [ ]:
%matplotlib inline
fig, ax = plt.subplots(num='Medium Slew', figsize=(20, 6))

ax.plot(times, azimuth, 'C0o-', 
        label='Azimuth Position', lw=0.5, zorder=3)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Azimuth Position [degrees]')
ax.set_title(f'Medium Slew Dome Rotation test')

ax2 = ax.twinx()
ax2.plot(times, speed_deriv, 'x', color = 'green',
                label='Azimuth Speed', lw=0.5)
ax2.set_ylabel('Azimuth Speed [deg/s]')
ax2.set_ylim(-4.5,4.5)
plt.hlines(y=[-4, 4], xmin=[times[0]], xmax=[times[-1]], colors='orange', 
           linestyles='--', lw=2, label='Minimum Dome Rotation Speed Requirement')
fig.legend()
plt.show()

The dome azimuth speeed (green) is smaller than the required 4 deg/s along the move (orange horizontal dotted lines). 
If we remove the acceleration/decceleration parts of the move and focus in the uniform motion:


In [ ]:
fig, ax = plt.subplots(num='Medium Slew', figsize=(20, 6))
[i1,i2] = [75,105]

ax.plot(times[i1:i2], azimuth[i1:i2], 'C0o-', 
        label='Azimuth Position', lw=0.5, zorder=3)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Azimuth Position [degrees]')
ax.set_title(f'Medium Slew Dome Rotation test - Focus on uniform motion')

ax2 = ax.twinx()
ax2.plot(times[i1:i2], speed_deriv[i1:i2], 'x', color = 'green',
                label='Azimuth Speed', lw=0.5)
ax2.set_ylabel('Azimuth Speed [deg/s]')
ax2.set_ylim(-5,5)
plt.hlines(y=[-4, 4], xmin=[times[i1]-2], xmax=[times[i2]+2], colors='orange', 
           linestyles='--', lw=2, label='Minimum Dome Rotation Speed Requirement')
fig.legend()
plt.show()

In [ ]:
max_speed_medslew = np.max(abs(speed_deriv[i1:i2]))
print(f'The maximum slew speed is {max_speed_medslew:0.2f} deg/s')

In [ ]:
# Linear regression
model = np.polyfit(times[i1:i2], azimuth[i1:i2], 1)
slope_medslew = model[0]
print(f'The average slew speed is {abs(slope_medslew):0.2f} deg/s')

## Save results

In [ ]:
# Save test results in a file (See file attached)
with open(filename, 'a') as f:
    f.write('\n{} test on {} \n'.format(test_type, time.asctime()))
    f.write('Start Time, End Time, Rotation angle (degrees), ' 
            'Rotational speed (degrees/sec) \n')
    f.write(f'{start_time.value}, {end_time.value},' 
            f'{round(rotation_angle,2)},{round(med_speed,2)} \n')
    f.write(f'The maximum slew speed is {max_speed_medslew:0.2f} deg/s \n')
    f.write(f'The average slew speed is {abs(slope_medslew):0.2f} deg/s \n')
f.close()
print(f'Start and end move times, rotation angle (degrees) and rotational speeds ' 
      f'(degrees/sec) are saved in {filename}')

# Test Short Rotation - 30 degrees

In [ ]:
# # Short rotation - 30 degrees
test_type = 'Short Rotation - 30 deg'
initial_position = 180
final_position = 210

print(f'The initial position is {initial_position} degrees \n'
      f'The final position is {final_position} degrees \n'
      f'Results will be saved in {filename}')

In [ ]:
 # Move to initial position 
print(f'Moving to initial azimuth position = '
      f'{initial_position} degrees on {time.asctime()}')

await atcs.slew_dome_to(az=initial_position)

# Get dome azimuth initial position and move start time 
start_time = Time(Time.now(), format='fits', 
                         scale='utc')

dome_azimuth = await client.select_top_n(
    "lsst.sal.ATDome.position", 'azimuthPosition', 1)
az_origin = dome_azimuth.iat[0,0]
print(f'Initial azimuth position is = {az_origin} degrees '
      f'on {start_time}')

# Move to final position
print(f'Moving to final azimuth position = '
      f'{final_position} degrees on {time.asctime()}')
await atcs.slew_dome_to(az=final_position)

# Get dome azimuth final position and move end time
end_time = Time(Time.now(), format='fits', scale='utc')

dome_azimuth = await client.select_top_n(
    "lsst.sal.ATDome.position", 'azimuthPosition', 1)
az_final = dome_azimuth.iat[0,0]
print(f'Final azimuth position is = {az_final} degrees ' 
      f'on {end_time}')


# Some quick calculations and print results. 
duration_time= TimeDelta(end_time - start_time, 
                         format='sec')
rotation_angle = abs(az_origin - az_final)

short_speed = rotation_angle/duration_time.value

## Results

In [ ]:
print(f'######################################################### \n'
          f'The move lasted {duration_time.value:.2f} seconds \n'
          f'for an angular rotation of {rotation_angle:.2f} degrees \n'
          f'with an average rotational speed of {short_speed:.2f} degrees/sec\n'
          f'#########################################################')
    

## Dome azimuth speed

In [ ]:
# Get EFD azimuth position time series between start_time and end_time
df_shortslew = await client.select_time_series("lsst.sal.ATDome.position", 'azimuthPosition', 
                                                    start_time, end_time)

In [ ]:
azimuth = np.array(df_shortslew.values.tolist())[:,0]
times=(df_shortslew.index - df_shortslew.index[0]).total_seconds()
speed_deriv = np.gradient(azimuth, times)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(num='Short Slew', figsize=(20, 6))

ax.plot(times, azimuth, 'C0o-', 
        label='Azimuth Position', lw=0.5, zorder=3)
# ax.grid('-', alpha=0.2)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Azimuth Position [degrees]')
ax.set_title(f'Short Slew Dome Rotation test')

ax2 = ax.twinx()
ax2.plot(times, speed_deriv, 'x', color = 'green',
                label='Azimuth Speed', lw=0.5)
ax2.set_ylabel('Azimuth Speed [deg/s]')
ax2.set_ylim(-4.5,4.5)
plt.hlines(y=[-4, 4], xmin=[times[0]], xmax=[times[-1]], colors='red', 
           linestyles='--', lw=2, label='Minimum Dome Rotation Speed Requirement')
fig.legend()
plt.show()

We can see that in this shorter slew, the dome speed (green) was increasing and decreasing and didn't stay in a cruising speed for long. But in any case, the speed also stay smaller than the required 4 deg/s. Below is the maximum azimuth speed of this rotation:

In [ ]:
max_speed_shortslew = np.max(abs(speed_deriv))
print(f'The maximum slew speed is {max_speed_shortslew:0.2f} deg/s')

## Save results

In [ ]:
# Save test results in a file (See file attached)
with open(filename, 'a') as f:
    f.write('\n{} test on {} \n'.format(test_type, time.asctime()))
    f.write('Start Time, End Time, Rotation angle (degrees), ' 
            'Rotational speed (degrees/sec) \n')
    f.write(f'{start_time.value}, {end_time.value},' 
            f'{round(rotation_angle,2)},{round(short_speed,2)} \n')
    f.write(f'The maximum slew speed is {max_speed_shortslew:0.2f} deg/s \n')
f.close()
print(f'Start and end move times, rotation angle (degrees) and rotational speeds ' 
      f'(degrees/sec) are saved in {filename}')

# Shutdown

In [ ]:
await atcs.home_dome()

In [ ]:
# Back in standby.
await atcs.standby()

# Offline analysis of observing run dome speed (2022-02-16) 

A graph of the dome speed during the night of 2022-02-16 shows that the maximum dome azimuth speed was never above 4 deg/sec as the LTS-337-030 (Auxiliary Telescope Dome Rotation) requires.

In [ ]:
# Night dome speed during 2022 February observing run. 

In [ ]:
# Get EFD azimuth position time series between start_time and end_time
start_time = Time('2022-02-16T00:00:01.00', scale = 'utc')
end_time = Time('2022-02-16T09:00:00.00', scale = 'utc')
df_nightslew = await client.select_time_series("lsst.sal.ATDome.position", 'azimuthPosition', 
                                                    start_time, end_time)

In [ ]:
azimuth = np.array(df_nightslew.values.tolist())[:,0]
times=(df_nightslew.index - df_nightslew.index[0]).total_seconds()
speed_deriv = np.gradient(azimuth, times)


In [ ]:
%matplotlib inline
fig, ax = plt.subplots(num='Night Slew', figsize=(20, 6))

ax.plot(times, azimuth, 'C0o-', 
        label='Azimuth Position', lw=0.5, zorder=3)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Azimuth Position [degrees]')
ax.set_title(f'Dome Position and Speed during night 2022-02-16')

ax2 = ax.twinx()
ax2.plot(times, speed_deriv, 'x', color = 'green',
                label='Azimuth Speed', lw=0.5)
ax2.set_ylabel('Azimuth Speed [deg/s]')
ax2.set_ylim(-4.5,4.5)
plt.hlines(y=[-4, 4], xmin=[times[0]], xmax=[times[-1]], 
           colors='orange', linestyles='--', lw=2, label='Minimum Dome Rotation Speed Requirement')
fig.legend()
plt.show()


In [ ]:
max_speed_allnightslew = np.max(abs(speed_deriv))
print(f'The maximum dome speed in the night 2022-02-16 was {max_speed_allnightslew:0.2f} deg/s')

In [ ]:
# Choosing a random slew

In [ ]:
# Get EFD azimuth position time series between start_time and end_time
start_time = Time('2022-02-16T05:12:19.138', scale = 'utc')
end_time = Time('2022-02-16T05:14:46.782', scale = 'utc')
df_randomslew = await client.select_time_series("lsst.sal.ATDome.position", 'azimuthPosition', 
                                                    start_time, end_time)

In [ ]:
azimuth = np.array(df_randomslew.values.tolist())[:,0]
times=(df_randomslew.index - df_randomslew.index[0]).total_seconds()
speed_deriv = np.gradient(azimuth, times)


In [ ]:
%matplotlib inline
fig, ax = plt.subplots(num='Random Slew', figsize=(20, 6))

ax.plot(times, azimuth, 'C0o-', 
        label='Azimuth Position', lw=0.5, zorder=3)
# ax.grid('-', alpha=0.2)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Azimuth Position [degrees]')
ax.set_title(f'Random Dome Slew during night 2022-02-16')

ax2 = ax.twinx()
ax2.plot(times, speed_deriv, 'x', color = 'green',
                label='Azimuth Speed', lw=0.5)
ax2.set_ylabel('Azimuth Speed [deg/s]')
ax2.set_ylim(-4.5,4.5)
#ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.hlines(y=[-4, 4], xmin=[times[0]], xmax=[times[-1]], 
           colors='orange', linestyles='--', lw=2, label='Minimum Dome Rotation Speed Requirement')
fig.legend()
plt.show()


In [ ]:
max_speed_randomslew = np.max(abs(speed_deriv))
print(f'The maximum speed during this random dome slew is {max_speed_randomslew:0.2f} deg/s')

****

# Conclusion

The dome azimuth rotation speed has been examined in 3 dedicated rotation tests of different rotation angles (210, 90 and 30 degrees) and an offline analysis of an observing night, to replicate the dome operations cadence.
In none of these test cases, the azimuth rotational speed has been above the minimum 4 deg/sec speed requirement from LTS-337-030 (Auxiliary Telescope Dome Rotation Requirement). 
